# Borrowers reliability research

The customer is the bank credit department. It is necessary to find out whether the clients marital status and the number of children affect the fact of repaying the loan on time. Input data from the bank - statistics on the solvency of customers.

The results of the research will be taken into account when building a **credit scoring** model - a special system that assesses the reliability of a potential borrower.

__Content:__

1. [Introduction to data](#start)
2. [Data preprocessing](#preprocessing)
 - [Gaps](#passes)
 - [Changing the data type](#types)
 - [Duplicates](#duplicates)
 - [Categorization](#categorization)
3. [Answers to the questions of the case](#main)
4. [Research results](#final)

<a id="start"></a>
## Introduction to data

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data.csv')

In [7]:
display(data.sample(5))
data.info()
data.describe().T

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16835,1,-1102.390881,49,high school,1,divorced,3,M,companion,0,251353.707728,for the purchase of a car
1639,0,375609.757769,65,high school,1,married,0,F,retired,0,68135.855633,buying a house for a family
13010,3,-2524.862867,35,high school,1,married,0,M,companion,0,292992.491992,purchase of a hous for rent
2863,0,363807.531549,65,elementary,3,married,0,F,retired,0,117137.352825,for the purchase of a car
12164,0,378174.187808,64,high school,1,married,0,M,retired,0,289338.622578,getting additional education


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


**Conclusion**

To answer the questions of the case, all df columns are not required. Columns `days_employed`, `dob_years`, `education`, `education_id`, `gender`, `familty_status` should be removed.

There are obvious data gaps in `total_income` column, that we are interested in

<a id="preprocessing"></a>
## Data preprocessing

Before we start, let's remove unnecessary columns

In [8]:
data = data.drop(columns = ['days_employed', 'dob_years', 'education', 'education_id', 'gender', 'family_status_id'])

<a id="passes"></a>
### Gaps

Let's look at the `days_employed` column

In [9]:
data[data['total_income'].isna()].head(10)

,children,family_status,income_type,debt,total_income,purpose
12,0,cohabitation,retired,0,NaN,play a wedding
26,0,married,civil servant,0,NaN,education
29,0,single,retired,0,NaN,construction of a hous
41,0,married,civil servant,0,NaN,used car deal
55,0,cohabitation,retired,1,NaN,play a wedding
65,0,single,companion,0,NaN,operations with commercial property
67,0,married,retired,0,NaN,buying a house for a family
72,1,married,civil servant,0,NaN,operations with commercial property
82,2,married,employee,0,NaN,hous
83,0,married,employee,0,NaN,hous


The NaN value in the `total_income` column is not the equivalent of "zero" because the `income_type` column contains employees and civil servants who cannot have no income at all. Perhaps it is an error. We need to fill empty values with the median or mean value. Consider both options and choose the most suitable.

In [10]:
data.groupby('income_type').agg({'total_income': ['mean','median']})

total_income               
                             mean         median
income_type                                     
civil servant       170898.309923  150447.935283
companion           202417.461462  172357.950966
employee            161380.260488  142594.396847
entrepreneur        499163.144947  499163.144947
on maternity leave   53829.130729   53829.130729
retired             137127.465690  118514.486412
student              98201.625314   98201.625314
unemployed          131339.751676  131339.751676

The median is either less than or equal to the mean. The latter most likely occurs in situations where there are at most two values in `income_type`. Let's check which types have missing values and count them.

In [11]:
data[data['total_income'].isna()]['income_type'].value_counts()

employee         1105
companion         508
retired           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

The category "entrepreneur" contains only one value. Judging by the median and average values, there are only two entrepreneurs, it is more logical to delete this rows than to fill them in. The rest will be filled with medians.

We will also add an `autofill_id` column to the data, where the value 1 will mean that the row has been modified. This is necessary for further processing of duplicates. The `.duplicated` method may treat such strings as duplicates, although they are not.

In [12]:
isna_income_types = data[data['total_income'].isna()]['income_type'].unique()

for income_type in isna_income_types:
    if income_type != 'entrepreneur':
        median = data[data['income_type'] == income_type]['total_income'].median()
        data.loc[(data['income_type'] == income_type) & (data['total_income'].isna()), 'autofill_id'] = 1        
        data.loc[(data['income_type'] == income_type) & (data['total_income'].isna()), 'total_income'] = median

data = data.dropna(subset = ['total_income']).reset_index(drop=True)

print('---Checking for the column autofill_id---')
display(data[data['autofill_id'] == 1].head(5))
print()
print('---Checking for empty values in a column total_income---')
print()
data.info()

---Checking for the column autofill_id---


,children,family_status,income_type,debt,total_income,purpose,autofill_id
12,0,cohabitation,retired,0,118514.486412,play a wedding,1.0
26,0,married,civil servant,0,150447.935283,education,1.0
29,0,single,retired,0,118514.486412,construction of a hous,1.0
41,0,married,civil servant,0,150447.935283,used car deal,1.0
55,0,cohabitation,retired,1,118514.486412,play a wedding,1.0



---Checking for empty values in a column total_income---

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   children       21524 non-null  int64  
 1   family_status  21524 non-null  object 
 2   income_type    21524 non-null  object 
 3   debt           21524 non-null  int64  
 4   total_income   21524 non-null  float64
 5   purpose        21524 non-null  object 
 6   autofill_id    2173 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 1.1+ MB


Now let's examine the rest of the columns for implicit gaps

In [13]:
print('--COL children--')
print(data['children'].value_counts())
print()
print('--COL family_status--')
print(data['family_status'].value_counts())
print()
print('--COL income_type--')
print(data['income_type'].value_counts())
print()
print('--COL debt--')
print(data['debt'].value_counts())
print()
print('--COL total_income values less than zero--')
print(data[data['total_income'] < 0]['total_income'].count())
print()
print('--COL purpose--')
print(data['purpose'].value_counts())

--COL children--
 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

--COL family_status--
married          12379
cohabitation      4177
single            2813
divorced          1195
widower/widow      960
Name: family_status, dtype: int64

--COL income_type--
employee              11119
companion              5085
retired                3856
civil servant          1459
unemployed                2
student                   1
entrepreneur              1
on maternity leave        1
Name: income_type, dtype: int64

--COL debt--
0    19783
1     1741
Name: debt, dtype: int64

--COL total_income values less than zero--
0

--COL purpose--
car                                    973
higher education                       879
wedding                                797
to hold a wedding                      777
play a wedding                         774
property transactions                  676
purchase of commerc

The `children` column has a -1 value, which is not possible because it specifies the number of children. Consider which `family_status` categories have the -1 values.

In [14]:
data[data['children'] < 0]['family_status'].value_counts()

married          29
cohabitation      5
single            5
divorced          4
widower/widow     4
Name: family_status, dtype: int64

Most of the values are among people who are married and approximately equally in other categories. This behavior of the data suggests rather that instead of -1 there should be a value of 1, the issue is only with people with the status "single / not married". Let's check how the children presence data is distributed in the ranges without errors.

In [15]:
data[data['children'] > 0]['family_status'].value_counts()

married          4851
cohabitation     1420
single            543
divorced          407
widower/widow     108
Name: family_status, dtype: int64

In the error-free range, people with single status also have children. The distribution of data in the considered range is similar to the distribution in the range with errors, with some minor exceptions. Most likely, the value -1 should be replaced with the value 1.

In [16]:
data['children'] = data['children'].replace(-1, 1)
data[data['children'] < 0]['family_status'].value_counts()

Series([], Name: family_status, dtype: int64)

**Conclusion**

- The `total_income` column had explicit gaps. They were filled with median values taking into account income categories
- Removed one line with an empty value in the `total_income` column, where `income_type` corresponded to the value "entrepreneur", this is less than 1% of the total data array, the result of the reserch should not be affected
- Errors were found in the values of the `children` column, where the value -1 is present. The behavior of data in a range with errors is roughly similar to the behavior of data in a range without errors. It was decided to replace the value -1 with 1.

<a id="types"></a>
### Changing the data type

Find out how much memory the current df consumes and compare this value based on the result of data type changes

In [17]:
data.info(verbose = False, memory_usage = 'deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Columns: 7 entries, children to autofill_id
dtypes: float64(2), int64(2), object(3)
memory usage: 4.9 MB


The values in the `total_income` column are the `float` data type. For our reserch, this is not necessary, and a lot of decimal places make it difficult to read the table. Let's get rid of it by changing the data type from `float` to `int`.

In [18]:
data['total_income'] = data['total_income'].astype('uint32')
data.head(5)

,children,family_status,income_type,debt,total_income,purpose,autofill_id
0,1,married,employee,0,253875,buying a house,NaN
1,1,married,employee,0,112080,purchasing a car,NaN
2,0,married,employee,0,145885,buying a house,NaN
3,3,married,employee,0,267628,additional education,NaN
4,0,cohabitation,retired,0,158616,play a wedding,NaN


For optimization purposes, it also makes sense to perform the following actions:
- Convert `children`, `debt` columns to `uint8` type
- `family_status`, `income_type` columns into `category` type

Let's do it

In [19]:
data['children'] = data['children'].astype('uint8')
data['debt'] = data['debt'].astype('uint8')
data['family_status'] = data['family_status'].astype('category')
data['income_type'] = data['income_type'].astype('category')

Now let's check data consumption after optimization

In [20]:
data.info(verbose = False, memory_usage = 'deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21524 entries, 0 to 21523
Columns: 7 entries, children to autofill_id
dtypes: category(2), float64(1), object(1), uint32(1), uint8(2)
memory usage: 1.9 MB


**Conclusion**

- Removed extra decimal places in the `total_income` column to make the data easier to perceive visually
- Changed the data type in the `total_income` column from `int64` to `uint32`
- Changed data type in `children` and `debt` columns from `int64` to `uint8`
- Changed data type in `family_status` and `income_type` columns from `object` to `category`
- Total memory usage reduced by 61% from 4.9 MB to 1.9 MB

<a id="duplicates"></a>
### Duplicates

Let's check the data for duplicates. To do this, we will use logical indexing, where the first condition will be the result of the `.duplicated` method, and the second condition will be a None value in the `autofill_id` column. Simply put, let's check if there are duplicates in those rows where we did not change the value of the `total_income` column at the gaps operating stage

In [21]:
duplicated_columns = data[(data.duplicated()) & (data['autofill_id'] != 1)].index.tolist()
duplicated_columns

[11532, 13258, 19477]

Three duplicates found. Let's get rid of them

In [22]:
data = data.drop(duplicated_columns).reset_index(drop = True)
data[(data.duplicated()) & (data['autofill_id'] != 1)]

,children,family_status,income_type,debt,total_income,purpose,autofill_id


**Conclusion**

- Using logical indexing we Checked the data for duplicates in the range that we did not change at the gaps operating stage
- Found three duplicates and found out their indices
- Get rid of them using the `.drop` method

<a id="categorization"></a>
### Categorization

At the implicit gaps operating stage, a lot of essentially same values were found in the `purpose` column. Let's create a function to categorize the data. It is proposed to use the following categories of loan purposes:
- Residential proprety for all offers where the word "hous" or "residential" is included (except for renovation)
- Other proprety for all offers where the word "proprety" is included (except for residential proprety and renovation)
- Renovation
- Wedding
- Education
- Car

In [23]:
def categorize(df):
    purpose = df['purpose']
    
    if 'renovation' in purpose:
        return 'renovation'
    elif 'hous' in purpose:
        return 'residential property'
    elif 'residential' in purpose:
        return 'residential property'
    elif 'property' in purpose:
        return 'other property'
    elif 'wedding' in purpose:
        return 'wedding'
    elif 'car' in purpose:
        return 'car'
    elif 'education' in purpose:
        return 'education'

    
data['p_cat'] = data.apply(categorize, axis=1)
data.p_cat.unique()

array(['residential property', 'car', 'education', 'wedding',
       'other property', 'renovation'], dtype=object)

In [24]:
data['purpose_category'] = data.apply(categorize, axis = 1)
data['purpose_category'].value_counts()

other property          5134
residential property    5093
car                     4314
education               4021
wedding                 2348
renovation               611
Name: purpose_category, dtype: int64

The following question was asked in the case: "Is there a relationship between having children and repaying the loan on time?". That is, we are interested in the presence or absence of children, but not their number. We need to enter two categories for the `children` column:
- Have children
- No children

In [25]:
def children_to_category(row):
    children = row['children']
    
    if children > 0:
        return 'have children'
    return 'no children'

In [26]:
data['children_category'] = data.apply(children_to_category, axis = 1)
data.head(10)

,children,family_status,income_type,debt,total_income,purpose,autofill_id,p_cat,purpose_category,children_category
0,1,married,employee,0,253875,buying a house,NaN,residential property,residential property,have children
1,1,married,employee,0,112080,purchasing a car,NaN,car,car,have children
2,0,married,employee,0,145885,buying a house,NaN,residential property,residential property,no children
3,3,married,employee,0,267628,additional education,NaN,education,education,have children
4,0,cohabitation,retired,0,158616,play a wedding,NaN,wedding,wedding,no children
5,0,cohabitation,companion,0,255763,buying a house,NaN,residential property,residential property,no children
6,0,married,companion,0,240525,hous operations,NaN,residential property,residential property,no children
7,0,married,employee,0,135823,education,NaN,education,education,no children
8,2,cohabitation,employee,0,95856,to hold a wedding,NaN,wedding,wedding,have children
9,0,married,employee,0,144425,buying a house for a family,NaN,residential property,residential property,no children


In [27]:
data[data['children_category'] == 'have children']['children'].unique()

array([ 1,  3,  2,  4, 20,  5], dtype=uint8)

In [28]:
data[data['children_category'] == 'no children']['children'].unique()

array([0], dtype=uint8)

The third question of the case is: "Is there a relationship between the level of income and repayment of the loan on time?"

For analysis, it is necessary to categorize the values of the `total_income` column. To do this, let's examine the column for key range values

In [29]:
print('Maximum income', data['total_income'].max())
print('Median income', data['total_income'].median())
print('Minimum income', data['total_income'].min())

Maximum income 2265604
Median income 142594.0
Minimum income 20667


Income fluctuates between ₽20k and ₽2.2m. The following categories are suggested:
- Up to ₽80k
- From ₽80k to ₽150k 
- From ₽150k to ₽250k 
- From ₽150k to ₽500k 
- From ₽500k to ₽1m
- Over ₽1m

In [30]:
def income_to_category(row):
    income = row['total_income']
    
    if income < 80000:
        return 'up to ₽80k'
    if income < 150000:
        return 'from ₽80k to ₽150k'
    if income < 250000:
        return 'from ₽150k to ₽250k'
    if income < 500000:
        return 'from ₽250k to ₽500k'
    if income < 1000000:
        return 'from ₽500k to ₽1m'
    return 'over ₽1m'

In [31]:
data['income_category'] = data.apply(income_to_category, axis = 1)
data

,children,family_status,income_type,debt,total_income,purpose,autofill_id,p_cat,purpose_category,children_category,income_category
0,1,married,employee,0,253875,buying a house,NaN,residential property,residential property,have children,from ₽250k to ₽500k
1,1,married,employee,0,112080,purchasing a car,NaN,car,car,have children,from ₽80k to ₽150k
2,0,married,employee,0,145885,buying a house,NaN,residential property,residential property,no children,from ₽80k to ₽150k
3,3,married,employee,0,267628,additional education,NaN,education,education,have children,from ₽250k to ₽500k
4,0,cohabitation,retired,0,158616,play a wedding,NaN,wedding,wedding,no children,from ₽150k to ₽250k
...,...,...,...,...,...,...,...,...,...,...,...
21516,1,cohabitation,companion,0,224791,hous operations,NaN,residential property,residential property,have children,from ₽150k to ₽250k
21517,0,married,retired,0,155999,car deal,NaN,car,car,no children,from ₽150k to ₽250k
21518,1,cohabitation,employee,1,89672,property,NaN,other property,other property,have children,from ₽80k to ₽150k
21519,3,married,employee,1,244093,to buy a car,NaN,car,car,have children,from ₽150k to ₽250k


In [32]:
data.groupby('income_category').agg({'total_income' : ['min', 'max']})

total_income         
                             min      max
income_category                          
from ₽150k to ₽250k       150001   249991
from ₽250k to ₽500k       250130   499924
from ₽500k to ₽1m         502318   997014
from ₽80k to ₽150k         80039   149993
over ₽1m                 1004476  2265604
up to ₽80k                 20667    79973

For good pivot table visualization, we also need to enter the verbal categories of the `debt` column, which now has 0 and 1 value, where, obviously, 0 corresponds to the absence of delays, and 1 to the presence. We use the following values:

- have debts
- no debts

In [33]:
def debt_to_category(row):
    debt = row['debt']
    if debt == 1:
        return 'have debts'
    return 'no debts'

data['debt_category'] = data.apply(debt_to_category, axis = 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21521 entries, 0 to 21520
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   children           21521 non-null  uint8   
 1   family_status      21521 non-null  category
 2   income_type        21521 non-null  category
 3   debt               21521 non-null  uint8   
 4   total_income       21521 non-null  uint32  
 5   purpose            21521 non-null  object  
 6   autofill_id        2173 non-null   float64 
 7   p_cat              21521 non-null  object  
 8   purpose_category   21521 non-null  object  
 9   children_category  21521 non-null  object  
 10  income_category    21521 non-null  object  
 11  debt_category      21521 non-null  object  
dtypes: category(2), float64(1), object(6), uint32(1), uint8(2)
memory usage: 1.3+ MB


**Conclusions**

To simplify the analysis, the following columns have been added and the following categories have been introduced:
1. `purpose_category` column to categorize the `purpose` column
- Residential proprety for all offers where the word "hous" or "residential" is included (except for renovation)
- Other proprety for all offers where the word "proprety" is included (except for residential proprety and renovation)
- Renovation
- Wedding
- Education
- Car
2. `children_category` column to categorize the `children` column with the following values:
- have children
- no children
3. `income_category` column to categorize the `total_income` column with the following values:
- Up to ₽80k
- From ₽80k to ₽150k
- From ₽150k to ₽250k
- From ₽150k to ₽500k
- From ₽500k to ₽1m
- Over ₽1m
4. `debt_category` column to categorize the `debt` column with the following values:
- have debts
- no debts

<a id="main"></a>
## Answers to the questions of the case

To answer the questions of the case, it is best to use pivot tables. To simplify this process, let's create a function that will create a pivot table for a specific categorical column.

In [40]:
def to_pivot(df, category):
    pivot = df.pivot_table(index = category, columns = 'debt_category', values = 'debt', aggfunc = 'count')
    pivot['total'] = pivot['have debts'] + pivot['no debts']
    pivot['% debtors'] = round(pivot['have debts'] / pivot['total'] * 100, 2)
    pivot = pivot.sort_values(by='% debtors', ascending=False)
    return pivot

Also, to answer the questions of the case, it is necessary to understand what is the average value of the share of debtors over the entire data sample

In [41]:
debt_ratio = data['debt'].sum() / data['debt'].count()
print('The average number of debtors in the entire data sample: {:.0%}'.format(debt_ratio))

The average number of debtors in the entire data sample: 8%


### Is there a relationship between having children and repaying the loan on time?

In [42]:
pivot_children = to_pivot(data, 'children_category')
pivot_children

debt_category,have debts,no debts,total,% debtors
children_category,,,,
have children,678,6698,7376,9.19
no children,1063,13082,14145,7.52


**Conclusion**

The value of the debtors share with children is on 1.2% higher than the average value, and the value of the debtors share without children is on 0.5% lower.

It can be unequivocally stated that borrowers with children repay loans less often than borrowers without children.

### Is there a relationship between marital status and loan repayment on time?

In [43]:
pivot_family = to_pivot(data, 'family_status')
pivot_family

debt_category,have debts,no debts,total,% debtors
family_status,,,,
single,274,2538,2812,9.74
cohabitation,388,3789,4177,9.29
married,931,11446,12377,7.52
divorced,85,1110,1195,7.11
widower/widow,63,897,960,6.56


**Вывод**

Most often, people who indicate their status as "single" and "cohabitation" are the most likely to overdue their loans - by 1.7% and 1.3%, respectively. Widowers are the least likely to overdue by 1.5%, and people who indicate the status of "divorced" and "married" are little less than the average fall into debt, by 0.9% and 0.5%, respectively.

Dependence is clearly present and it is not obvious.

### Is there a relationship between income level and loan repayment on time?

In [44]:
pivot_income = to_pivot(data, 'income_category')
pivot_income

debt_category,have debts,no debts,total,% debtors
income_category,,,,
from ₽80k to ₽150k,804,8605,9409,8.55
from ₽150k to ₽250k,569,6456,7025,8.10
over ₽1m,2,23,25,8.00
up to ₽80k,174,2101,2275,7.65
from ₽250k to ₽500k,180,2410,2590,6.95
from ₽500k to ₽1m,12,185,197,6.09


**Conclusion**

people with incomes from 500k to 1m Least of all allow loan delays, by as much as 2%. People with incomes up to 80k and in the range from 250k to 500k fall into debt less than the average by 0.4% and 1.1%, respectively.

People with incomes from 80,000 to 150,000 most often delay loans by 0.5%, and people in the category from 150,000 to 250,000 and more than 1m rubles are similar to avarage value.

There is rather no dependence, fluctuations in indicators are insignificant.

## How do different purposes of a loan affect its repayment on time?

In [45]:
pivot_purpose = to_pivot(data, 'purpose_category')
pivot_purpose

debt_category,have debts,no debts,total,% debtors
purpose_category,,,,
car,403,3911,4314,9.34
education,370,3651,4021,9.20
wedding,186,2162,2348,7.92
other property,385,4749,5134,7.50
residential property,362,4731,5093,7.11
renovation,35,576,611,5.73


**Conclusion**

People who take money for car and education perposes are most often allow loan delays by 1.3% and 1.2%, respectively. Least get into debtors are people who make various real estate transactions. A record low value is noted in the indicator "repair" - 5.7% against 8% of the average.

People who take money for a wedding are in the middle range.

<a id="final"></a>
## Research results

The studied data sample had obvious gaps (about 10% of the total amount of information) in the `total_income` column. The gaps were filled in with the median values subject to the categories of the `income_type` column. In the process of data preprocessing, only 4 rows were removed, three of which turned out to be duplicates.

For the convenience of analytical work, the data were categorized as follows:

1. `purpose_category` column to categorize the `purpose` column
- Residential proprety for all offers where the word "hous" or "residential" is included (except for renovation)
- Other proprety for all offers where the word "proprety" is included (except for residential proprety and renovation)
- Renovation
- Wedding
- Education
- Car
2. `children_category` column to categorize the `children` column with the following values:
- have children
- no children
3. `income_category` column to categorize the `total_income` column with the following values:
- Up to ₽80k
- From ₽80k to ₽150k
- From ₽150k to ₽250k
- From ₽150k to ₽500k
- From ₽500k to ₽1m
- Over ₽1m
4. `debt_category` column to categorize the `debt` column with the following values:
- have debts
- no debts

**8%** - is the average debtors percentage over the entire data sample

The analysis was carried out using the pivot table function, the result can be seen in [part 3](#main)

**As a result of data analysis, we can say the following:**

Deputies with children who are not married or in a cohabitation, as well as those who take loans for operations with a car and for education, more often do not repay loans on time than the rest by about 12-20%.

Divorced people, widowers and those who take out loans for home renovations are more likely to repay loans on time, this conclusion may be affected by a relatively small sample, a reserch on a larger sample is required.

The indicator of the income level rather does not or only slightly affects the repaying the loan on time probability.